In [1]:
import pyforest 
import requests
from bs4 import BeautifulSoup

In [2]:
%%bash 
cat 'project notes'

Brainstorming : 

1) Give user the option of choosing the country and job type. Will have to string format the link 
2) Some of the jobs have incomplete links. Will have to concat to make the link complete. Use regex and for loop to check if starts with https.
3) Will have to set a time between scrape for each page and then for scraping of each link. Have to set cap at number of total jobs as well. 
4) Ask the user for the list of skills he want a summary. That way we can expand it to incorporate any job title and skill set. 


Final Workflow:

1) Ask the user to input the following ; Job Title , Country, Skills Set
2) Write a small script that gives back how many jobs on indeed.
3) Ask the user to input how many jobs he wants to look through. Give a heads up that the more the jobs, the more the time it will take to run. Remember jobs are sorted by relevance by default.
4) Start the script.

In [3]:
import contextlib
import time

@contextlib.contextmanager
def timer():
    '''Calculate time it takes for process to complete
    
    Args:
      None
      
    Yields:
       float : the time in minutes for process to run
    '''
    start_time = time.time()
    
    yield
    
    end_time = time.time()
    
    final_time = (end_time - start_time) / 60
    
    print("The time it took to scrape and compile the results was : {:.2f} minutes".format(final_time))
    

In [84]:
from random import randint

print('Hello and welcome to the skills summariser. Let\'s get going')
print('')

job_title = input('Enter the job title you want to search for:')
country = input('Which country would you like to search in (no abbrevations please):')
skills = input('Enter the skills you want a summary for and make sure you seperate each skill by a space:')
n_jobs = input('How many jobs do you want to look through. The more the jobs the longer the script might take:')
#initial cleaning 

job_title = job_title.title()
country = country.title()
skills = skills.lower().split()

#'https://www.indeed.com/jobs?q=business+analyst&l=United+States'
primary_link = 'https://www.indeed.com/jobs?q={0}&l={1}'
further_pages_link = 'https://www.indeed.com/jobs?q={0}&l={1}&start={2}'


with timer():
    job_df = pd.DataFrame(columns = ['Job Title', 'Organization', 'Job Link'])
    num = 2
    start_range = list(range(10,50,10))
    skills_dict = {}
    titles = []
    orgs = []
    clean_links = []

    
    
#     while job_df.shape[0] <= int(n_jobs):
    if num == 1:
        link = primary_link.format(job_title,country)
        job_links_page = requests.get(link).text
        first_page_parser = BeautifulSoup(job_links_page,'html.parser')
        scraped_first_page = first_page_parser.body.find_all('div', attrs = {'data-tn-component':'organicJob'})
            
        for job in scraped_first_page:
            title = job.a.text.strip()
            titles.append(title)
    
            org = job.div.span.text.strip()
            orgs.append(org)
    
            link = job.a.get('href')
    
            if link.startswith('https'):
                clean_links.append(link)
            else:
                join_link = "https://www.indeed.com"+link
                clean_links.append(join_link)
            
        job_df['Job Title'] = titles
        job_df['Organization'] = orgs
        job_df['Job Link'] = clean_links
            
        for l in clean_links:
            try:
                jd_request = requests.get(l)
                if jd_request.status_code == 200:
                    jd_cont = jd_request.text
                    job_page = BeautifulSoup(jd_cont,'html.parser')
                    description = job_page.find('div', class_ = 'jobsearch-jobDescriptionText').text
                    description = description.lower()
        
                    for skill in skills:
                
                        if skill in description:
                            if skill in skills_dict:
                                skills_dict[skill] +=1
                            else:
                                skills_dict[skill] = 1
                        
        
        
        
            except:
                continue
        
        
#the code for scraping page 2 and forward starts from here
            
    else:
        
        temp_titles = []
        temp_orgs = []
        temp_links = []
        
        for num in start_range:
            link_cont = further_pages_link.format(job_title,country,num)
            more_links_page = requests.get(link_cont).text
            pages_parser = BeautifulSoup(more_links_page,'html.parser')
            scraped_page = pages_parser.body.find_all('div', attrs = {'data-tn-component':'organicJob'})
            
            for jobs in scraped_page:
                more_titles = jobs.a.text.strip()
                temp_titles.append(more_titles)
    
                more_orgs = jobs.div.span.text.strip()
                temp_orgs.append(more_orgs)
    
                more_links = jobs.a.get('href')
    
                if more_links.startswith('https'):
                    temp_links.append(more_links)
                else:
                    more_links = "https://www.indeed.com"+more_links
                    temp_links.append(more_links)
            
                job_df = job_df.append({'Job Title':more_titles,'Organization': more_orgs, 'Job Link':more_links}, ignore_index = True)
            
            

            
            for l in temp_links:
                try:
                    jd_request = requests.get(l)
                    if jd_request.status_code == 200:
                        jd_cont = jd_request.text
                        job_page = BeautifulSoup(jd_cont,'html.parser')
                        description = job_page.find('div', class_ = 'jobsearch-jobDescriptionText').text
                        description = description.lower()
        
                        for skill in skills:
                
                            if skill in description:
                                if skill in skills_dict:
                                    skills_dict[skill] += 1
                                else:
                                    skills_dict[skill] = 1
                except:
                    continue
            
        
        
        


Hello and welcome to the skills summariser. Let's get going

Enter the job title you want to search for:data analyst 
Which country would you like to search in (no abbrevations please):united states
Enter the skills you want a summary for and make sure you seperate each skill by a space:python sql excel tableau 
How many jobs do you want to look through. The more the jobs the longer the script might take:3


<IPython.core.display.Javascript object>

The time it took to scrape and compile the results was : 0.83 minutes


In [59]:
skills_dict

{'python': 2, 'excel': 6, 'tableau': 1, 'sql': 3, 'bash': 1}

In [70]:
skills_dict

{'excel': 83, 'python': 30, 'sql': 73, 'tableau': 12}

In [85]:
job_df

,Job Title,Organization,Job Link
0,Entry Level Data Analyst,Boiling Point,https://www.indeed.com/company/Boiling-Point/j...
1,Entry Level Business Analyst,Otcmarketpro,https://www.indeed.com/company/Otcmarketpro/jo...
2,Clinical Data Analyst,"WithHealth, Inc.","https://www.indeed.com/company/WithHealth,-Inc..."
3,Healthcare Data Analyst I,Inland Empire Health Plans,https://www.indeed.com/rc/clk?jk=323f01ad9a8ba...
4,"Data Analyst, Publishing Channels",Unity Technologies,https://www.indeed.com/rc/clk?jk=31a82ba0e6201...
5,Data Analyst - Temporary,"Refuse Specialists, LLC",https://www.indeed.com/company/Refuse-Speciali...
6,SQL Data Analyst,Peopleforce INC,https://www.indeed.com/company/RCI-technology-...
7,"Analyst, Data",Molina Healthcare,https://www.indeed.com/rc/clk?jk=108f29bf7f628...
8,Data Analyst - Teem,iOFFICE,https://www.indeed.com/rc/clk?jk=0428cc5997362...
9,WooCommerce Data Analyst,LiveGlam,https://www.indeed.com/rc/clk?jk=5896587b4d515...
